---
## Helpful Comments & Instructions
---

In [ ]:
# This file is a example/template for the `read_data.py` script, that loads the
# data from a specific paper and reformats it into the standard format used by
# SPAG. To use this, copy the following function into a testing script or
# Jupyter notebook, and modify the function name and contents as needed.


# Specifically, change all instances of...

# - `mittal2025` to the first author's last name (maybe with their first
# initial of there is a duplicate last name) and year of the paper. When there
# are instances of an author releasing more than one paper in a year, append a
# letter to the year in `x` position.
#   (e.g., `Ji et al. 2016a` --> `ji2016a`, if there is also a
#       `Ji et al. 2016b`)
#   (e.g., `Hansen et al. 2018` --> `hansent2018c`, if there are multiple
#       authors with the same last name and this was T. Hansen's 3rd paper in
#       2018)

# - the path to the data files to point to the correct location of the data
#   tables.
#   (e.g., `data_dir + "abundance_tables/mittal2025/table1.csv"`)

# - The `Reference` and `Ref` columns to match the citation for the paper.
#   --> `Reference` column should be in the form 'Author+Year'.
#       (e.g., 'Ji+2016a', 'HansenT+2018c')
#   --> `Ref` column should shortened form of the reference, typically the
#       first 3 letters of the author's last name (all uppercase) and the last
#       two digits of the year, with a letter appended if there are multiple
#       papers by the same author(s) in that year. (e.g.'JI16a', 'HANt18c')

# - The `Loc` column to match the type of object the stars are in. Use:
#   --> 'HA' for halo stars
#   --> 'BU' for bulge stars
#   --> 'DS' for disk stars
#   --> 'DW' for dwarf galaxy stars
#   --> 'UF' for ultra-faint dwarf galaxy stars
#   --> 'GC' for globular cluster stars

# - The `System` column to match the name of the system the stars are in, if
#   applicable and not already in the data table.


# Beyond this, confirm that the datafiles are reading in correctly, and that
# the columns are being filled in properly. You may need to modify how the data
# is read in or the structure of the datafile themselves, depending on the
# format of the data tables.

# Things to be wary of...

# - Some papers use different formats for the species names, such as "Fe I" vs
#   "FeI" vs "Fe1". The `ion_to_species()` function should be able to handle
#   most of these, but you may need to modify the datafile if there are any
#   issues, or contact A. Yelland.

# - Some papers may not provide all the necessary columns, such as separate
#   upper/lower limit flag column. You may need to modify the datafile to add
#   the flag columns.

# - Additionally, different authors at different times may use different solar
#   abundance scales. The `get_solar()` function can be used to retrieve the
#   solar abundances of the most commonly used scales, but if one is not
#   available, please let A. Yelland know.


---
## Package Imports & Directories
---

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import (division, print_function, absolute_import,
                        unicode_literals)
import  sys, os, glob, time, IPython

import astropy.constants as const
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
# from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, EarthLocation

# from PyAstronomy import pyasl

import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

import numpy as np
import scipy as sp
import pandas as pd

import seaborn as sns
sns.set_palette("colorblind")
colors = sns.color_palette("colorblind", 20)

# from smh import Session

from spag.read_data import *
from spag.convert import *
from spag.utils import *
# from spag.calculate import *
# import spag.read_data as rd
import spag.coordinates as scoord

# import alexmods.read_data as rd

script_dir = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]) + "/"
# script_dir = os.path.dirname(os.path.realpath(__file__))+"/"
data_dir = "C:/Users/Jacky Li/School Stuff Main/sophomore fall/Stars UROP/spag/spag/data/"
# plotting_dir = script_dir+"plots/"
# if not os.path.exists(plotting_dir):
#     os.makedirs(plotting_dir)


---
## Template Function
---

In [2]:

def df_reformatter(csv_df):
    '''
    For csv_files that are formatted like
    "Name, Num-X,loge(X),[X/H],e_[X/H],"
    then:
        Num-X, loge(X), [X/Fe], e_[X/Fe],

    Convert to :
    Name, Species, Nlines, logepsX, [X/H], e_[X/H], [X/Fe], e_[X/Fe]

    Returns an edited DF

    '''

    def get_species_name(numx_col_index):
        '''
        Get species name from the Num-X column
        '''

        num_x_str = csv_df.columns[numx_col_index]

        # species = num_x_str[5:] # remove Num-

        # if len(species) > 2:
        #     return species[1:3] + " " + species[3:]
        # else:
        #     return species

        # test_col = "Num-FeII"


        if "Num-" in num_x_str:
            col = num_x_str.replace("Num-","")

            if "II" in col[-2:]:
                elem = col[:-2] + " " + col[-2:]

            elif "I" in col[-1:]:
                elem = col[:-1] + " " + col[-1:]
            else:
                suffix  = get_default_ion(col)
                suffix2 = int_to_roman(suffix)
                elem = f"{col} {suffix2}"
            return elem


    ## Create the df with the correct columns
    # new_df = pd.DataFrame(columns=['Name', 'Species', 'Nlines', 'logepsX', '[X/H]', 'e_[X/H]', '[X/Fe]', 'e_[X/Fe]'])
    new_df = pd.DataFrame(columns=['Name','Species','Nlines','l_logepsX','logepsX','l_[X/H]','[X/H]','e_[X/H]','l_[X/Fe]','[X/Fe]',"e_[X/Fe]",'e_logepsX',])

    ## First pass: Get the Fe/ hydrogen values, and create a new row.

    for num_lines_col_index in [1,5]:
        for index, row_data in csv_df.iterrows():

            name = row_data.iloc[0]
            species = get_species_name(num_lines_col_index)
            nlines = row_data.iloc[num_lines_col_index]
            logepsx = row_data.iloc[num_lines_col_index+1]
            X_H = row_data.iloc[num_lines_col_index+2]
            eX_H = row_data.iloc[num_lines_col_index+3]
            X_Fe = np.nan
            eX_Fe = np.nan


            next_row  = {
                'Name':name, 'Species':species, 'Nlines':nlines, 'logepsX':logepsx,
                '[X/H]': X_H, 'e_[X/H]': eX_H, '[X/Fe]': X_Fe, 'e_[X/Fe]':eX_Fe
            }

            # Add to new_Df
            next_row_df = pd.DataFrame([next_row], columns=next_row.keys())

            new_df = pd.concat([new_df,next_row_df])



    ## second pass:
    # Get the over FE values, and create a new row.
    # Skip the over hydrogen values
    # Repeat until the all columns have been reached.


    for num_lines_col_index in range(9,len(csv_df.columns),4):
        for index, row_data in csv_df.iterrows():

            name = row_data.iloc[0]
            species = get_species_name(num_lines_col_index)
            nlines = row_data.iloc[num_lines_col_index]
            logepsx = row_data.iloc[num_lines_col_index+1]
            X_H = np.nan
            eX_H = np.nan
            X_Fe =  row_data.iloc[num_lines_col_index+2]
            eX_Fe = row_data.iloc[num_lines_col_index+3]


            # next_row = [ name, species, nlines, logepsx,
            #           X_H, eX_H, X_Fe, eX_Fe]

            next_row  = {
                'Name':name, 'Species':species, 'Nlines':nlines, 'logepsX':logepsx,
                '[X/H]': X_H, 'e_[X/H]': eX_H, '[X/Fe]': X_Fe, 'e_[X/Fe]':eX_Fe
            }

            # Add to new_Df
            next_row_df = pd.DataFrame([next_row], columns=next_row.keys())

            new_df = pd.concat([new_df,next_row_df])


    return new_df


def load_mittal2025(io=None):
    """
    Load the Author et al. YYYYx data for the <grouping or classification of stars>.

    Table 1 - Observations
    Table 2 - Stellar Parameters
    Table 3 - Abundance Table
    """

    ## Read in the data tables
    obs_df = pd.read_csv(data_dir + "abundance_tables/mittal2025/ob.csv", comment="#", na_values=["", " ", "nan", "NaN", "N/A", "n/a"])
    param_df = pd.read_csv(data_dir + "abundance_tables/mittal2025/table3_param.csv", comment="#", na_values=["", " ", "nan", "NaN", "N/A", "n/a"])
    abund_df = pd.read_csv(data_dir + "abundance_tables/mittal2025/table4_abun.csv", comment="#", na_values=["", " ", "nan", "NaN", "N/A", "n/a"])
    abund_df = df_reformatter(abund_df)
    display(abund_df)

    ## Make the new column names    ## Make the new column names
    species = []
    for ion in abund_df["Species"].unique():
        species_i = ion_to_species(ion)
        elem_i = ion_to_element(ion)
        if species_i not in species:
            species.append(species_i)

    epscols = [make_epscol(s) for s in species]
    ulcols = [make_ulcol(s) for s in species]
    XHcols = [make_XHcol(s).replace(" ", "") for s in species]
    ulXHcols = ['ul' + col for col in XHcols]
    XFecols = [make_XFecol(s).replace(" ", "") for s in species]
    ulXFecols = ['ul' + col for col in XFecols]
    errcols = [make_errcol(s) for s in species]

    ## New dataframe with proper columns
    authorYYYYx_df = pd.DataFrame(
                    columns=['I/O','Name','Simbad_Identifier','Reference','Ref','Loc','System','RA_hms','RA_deg','DEC_dms','DEC_deg',
                    'Teff','logg','Fe/H','Vmic'] + epscols + ulcols + XHcols + ulXHcols + XFecols
                    + ulXFecols + errcols)
    for i, name in enumerate(abund_df['Name'].unique()):
        authorYYYYx_df.loc[i,'Name'] = name
        authorYYYYx_df.loc[i,'Simbad_Identifier'] = obs_df.loc[obs_df['Name'] == name, 'Simbad_Identifier'].values[0]
        authorYYYYx_df.loc[i,'Reference'] = 'Mittal+2025'
        authorYYYYx_df.loc[i,'Ref'] = 'AUTYYx'
        authorYYYYx_df.loc[i,'I/O'] = 1
        authorYYYYx_df.loc[i,'Loc'] = 'UF' # [HA, BU, DS, DW, UF, GC]
        authorYYYYx_df.loc[i,'System'] = np.nan #obs_df.loc[obs_df['Name'] == name, 'System'].values[0]
        authorYYYYx_df.loc[i,'RA_hms'] = obs_df.loc[obs_df['Name'] == name, 'RA_hms'].values[0]
        authorYYYYx_df.loc[i,'RA_deg'] = scoord.ra_hms_to_deg(authorYYYYx_df.loc[i,'RA_hms'], precision=6)
        authorYYYYx_df.loc[i,'DEC_dms'] = obs_df.loc[obs_df['Name'] == name, 'DEC_dms'].values[0]
        authorYYYYx_df.loc[i,'DEC_deg'] = scoord.dec_dms_to_deg(authorYYYYx_df.loc[i,'DEC_dms'], precision=2)
        authorYYYYx_df.loc[i,'Teff'] = param_df.loc[param_df['Name'] == name, 'Teff'].values[0]
        authorYYYYx_df.loc[i,'logg'] = param_df.loc[param_df['Name'] == name, 'logg'].values[0]
        authorYYYYx_df.loc[i,'M/H'] = param_df.loc[param_df['Name'] == name, 'M/H'].values[0]
        authorYYYYx_df.loc[i,'Vmic'] = param_df.loc[param_df['Name'] == name, 'Vmic'].values[0]

        ## Fill in data
        star_df = abund_df[abund_df['Name'] == name]
        for j, row in star_df.iterrows():
            ion = row["Species"]
            species_i = ion_to_species(ion)
            elem_i = ion_to_element(ion)

            logepsX_sun_a09 = get_solar(elem_i, version='asplund2009')[0]
            logepsFe_a09 = star_df.loc[star_df['Species'] == 'Fe I', 'logepsX'].values[0]
            feh_a09 = logepsFe_a09 - get_solar('Fe', version='asplund2009')[0]

            ## Assign epsX values
            col = make_epscol(species_i)
            if col in epscols:
                authorYYYYx_df.loc[i, col] = row["logepsX"] if pd.isna(row["l_logepsX"]) else np.nan

            ## Assign ulX values
            col = make_ulcol(species_i)
            if col in ulcols:
                authorYYYYx_df.loc[i, col] = row["logepsX"] if pd.notna(row["l_logepsX"]) else np.nan

            ## Assign [X/H] and ul[X/H]values
            col = make_XHcol(species_i).replace(" ", "")
            if col in XHcols:
                if pd.isna(row["l_[X/H]"]):
                    authorYYYYx_df.loc[i, col] = normal_round(row["logepsX"] - logepsX_sun_a09, 2)
                    authorYYYYx_df.loc[i, 'ul'+col] = np.nan
                else:
                    authorYYYYx_df.loc[i, col] = np.nan
                    authorYYYYx_df.loc[i, 'ul'+col] = normal_round(row["logepsX"] - logepsX_sun_a09, 2)
                if 'e_[X/H]' in row.index:
                    authorYYYYx_df.loc[i, 'e_'+col] = row["e_[X/H]"]

            ## Assign [X/Fe] values
            col = make_XFecol(species_i).replace(" ", "")
            if col in XFecols:
                if pd.isna(row["l_[X/Fe]"]):
                    authorYYYYx_df.loc[i, col] = normal_round((row["logepsX"] - logepsX_sun_a09) - feh_a09, 2)
                    authorYYYYx_df.loc[i, 'ul'+col] = np.nan
                else:
                    authorYYYYx_df.loc[i, col] = np.nan
                    authorYYYYx_df.loc[i, 'ul'+col] = normal_round((row["logepsX"] - logepsX_sun_a09) - feh_a09, 2)
                if 'e_[X/Fe]' in row.index:
                    authorYYYYx_df.loc[i, 'e_'+col] = row["e_[X/Fe]"]

            ## Assign error values
            col = make_errcol(species_i)
            if col in errcols:
                e_logepsX = row.get("e_logepsX", np.nan)
                if pd.notna(e_logepsX):
                    authorYYYYx_df.loc[i, col] = e_logepsX
                else:
                    authorYYYYx_df.loc[i, col] = np.nan

    ## Drop the Fe/Fe columns
    authorYYYYx_df.drop(columns=[col for col in authorYYYYx_df.columns if 'Fe/Fe' in col or 'Fe2/Fe' in col], inplace=True, errors='ignore')

    return authorYYYYx_df


df = load_mittal2025()
display(df)

C:\Users\Jacky Li\AppData\Local\Temp\ipykernel_33204\270405097.py:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df,next_row_df])
C:\Users\Jacky Li\AppData\Local\Temp\ipykernel_33204\270405097.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df,next_row_df])


,Name,Species,Nlines,l_logepsX,logepsX,l_[X/H],[X/H],e_[X/H],l_[X/Fe],[X/Fe],e_[X/Fe],e_logepsX
0,BD+10 2495,Fe I,81,NaN,5.54,NaN,-1.96,0.05,NaN,NaN,NaN,NaN
0,BD+19 1185A,Fe I,54,NaN,6.34,NaN,-1.16,0.05,NaN,NaN,NaN,NaN
0,BD+24 1676,Fe I,85,NaN,5.10,NaN,-2.40,0.05,NaN,NaN,NaN,NaN
0,BD+26 3578,Fe I,76,NaN,5.35,NaN,-2.15,0.05,NaN,NaN,NaN,NaN
0,BD+29 2356,Fe I,87,NaN,5.97,NaN,-1.53,0.05,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
0,HE 0938+0114,Zn I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,HE 1012-1540,Zn I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,HE 1124-2335,Zn I,2.0,NaN,2.07,NaN,NaN,NaN,NaN,0.51,0.27,NaN
0,HE 1320-1339,Zn I,3.0,NaN,2.20,NaN,NaN,NaN,NaN,0.31,0.13,NaN


C:\Users\Jacky Li\AppData\Local\Temp\ipykernel_33204\270405097.py:175: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  logepsX_sun_a09 = get_solar(elem_i, version='asplund2009')[0]
C:\Users\Jacky Li\AppData\Local\Temp\ipykernel_33204\270405097.py:177: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  feh_a09 = logepsFe_a09 - get_solar('Fe', version='asplund2009')[0]
C:\Users\Jacky Li\AppData\Local\Temp\ipykernel_33204\270405097.py:175: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access

,I/O,Name,Simbad_Identifier,Reference,Ref,Loc,System,RA_hms,RA_deg,DEC_dms,...,e_[Ti/H],e_[Ti/Fe],e_[Cr/H],e_[Cr/Fe],e_[Cr2/H],e_[Cr2/Fe],e_[Ni/H],e_[Ni/Fe],e_[Zn/H],e_[Zn/Fe]
0,1,BD+10 2495,BD+10 2495,Mittal+2025,AUTYYx,UF,NaN,12:59:19.9306,194.833044,+09:14:35.634,...,NaN,0.13,NaN,0.07,NaN,0.19,NaN,0.08,NaN,0.21
1,1,BD+19 1185A,HD 250792,Mittal+2025,AUTYYx,UF,NaN,06:03:14.8636,90.811932,+19:21:38.696,...,NaN,0.11,NaN,0.09,NaN,0.20,NaN,0.08,NaN,0.14
2,1,BD+24 1676,BD+24 1676,Mittal+2025,AUTYYx,UF,NaN,07:30:41.2662,112.671943,+24:05:10.248,...,NaN,0.12,NaN,0.11,NaN,0.27,NaN,0.13,NaN,0.27
3,1,BD+26 3578,HD 338529,Mittal+2025,AUTYYx,UF,NaN,19:32:31.9104,293.13296,+26:23:26.117,...,NaN,0.13,NaN,0.12,NaN,0.35,NaN,0.16,NaN,0.26
4,1,BD+29 2356,BD+29 2356,Mittal+2025,AUTYYx,UF,NaN,13:01:52.4291,195.468455,+29:11:17.936,...,NaN,0.15,NaN,0.10,NaN,0.32,NaN,0.09,NaN,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,1,HE 0938+0114,Ross 889,Mittal+2025,AUTYYx,UF,NaN,09:40:43.2053,145.180022,+01:00:29.511,...,NaN,0.13,NaN,0.13,NaN,NaN,NaN,0.20,NaN,NaN
307,1,HE 1012-1540,HE 1012-1540,Mittal+2025,AUTYYx,UF,NaN,10:14:53.4753,153.722814,-15:55:53.172,...,NaN,0.21,NaN,0.40,NaN,NaN,NaN,0.19,NaN,NaN
308,1,HE 1124-2335,HE 1124-2335,Mittal+2025,AUTYYx,UF,NaN,11:27:26.9490,171.862288,-23:52:05.593,...,NaN,0.17,NaN,0.12,NaN,NaN,NaN,0.15,NaN,0.27
309,1,HE 1320-1339,BD-13 3695,Mittal+2025,AUTYYx,UF,NaN,13:22:44.1122,200.683801,-13:55:31.431,...,NaN,0.11,NaN,0.07,NaN,0.21,NaN,0.08,NaN,0.13


---
## Testing Zone
---

In [ ]:
# test_col = "Num-FeII"

# if "Num-" in test_col:
#     col = test_col.replace("Num-","")
#     if "II" in col[-2:]:
#         elem = col[:-2] + " " + col[-2:]
#     elif "I" in col[-1:]:
#         elem = col[:-1] + " " + col[-1:]
#     else:
#         elem = col
#     print(elem)


# print(get_default_ion(" O"))
# print(getelem("O"))


'O'


---
## Helpful Conversion Scripts for Different Datafiles & Numbers
---

### Abundance Calculations

In [10]:
## Quick Abundance Conversions

### Solar Abundances
epsfe_sun = rd.get_solar('Fe')[0]
epsba_sun = rd.get_solar('Ba')[0]
epssr_sun = rd.get_solar('Sr')[0]

### From Anna's notes of a presentation
srfe = -1.15
epsba = -3.5
feh = -5.62

### Calculated log(eps) values
srh = srfe + feh
epssr = epssr_sun + srh
epsfe = epsfe_sun + feh

feh_new = epsfe - epsfe_sun
srh_new = epssr - epssr_sun
bah_new = epsba - epsba_sun

srfe_new = srh_new - feh_new
bafe_new = bah_new - feh_new

print(f"logeps(Fe):  {epsfe:.2f}")
print(f"logeps(Sr): <{epssr:.2f}")
print(f"logeps(Ba):  {epsba:.2f}")
print(f"[Fe/H]:      {feh_new:.2f}")
print(f"[Sr/H]:     <{srh_new:.2f}")
print(f"[Ba/H]:      {bah_new:.2f}")
print(f"[Sr/Fe]:    <{srfe_new:.2f}")
print(f"[Ba/Fe]:     {bafe_new:.2f}")

NameError: name 'rd' is not defined

In [4]:
XH_from_eps(0.73, 'Sr')

-2.14

### CDS $\rightarrow$ CSV

In [ ]:
## Convert all CDS Standard Formatted Tables to CSVs for Stellar Abundances
create_csv_files = True
if create_csv_files:
    system =  'cayrel2004'
    table_numbers = [2,3,8]
    for n in table_numbers:

        base_path = f"/Users/ayelland/Research/metal-poor-stars/spag/spag/data_templates/cayrel2004/"
        table_path = base_path + f"table{n}.dat"
        readme_path = base_path + f"ReadMe"
        csv_path = base_path + f"table{n}.csv"

        if os.path.exists(table_path) and os.path.exists(readme_path):
            table_data = ascii.read(table_path, format='cds', readme=readme_path)
                    # guess=False,
                    # fast_reader=False

            table_data = table_data.to_pandas()
            table_data.to_csv(csv_path, index=False)
        else:
            print(f"Table {n} or ReadMe file does not exist for {system}.")


In [ ]:
## Convert CDS Standard Formatted Tables to CSVs for Galaxy Properties

from astropy.io import ascii
import os

create_csv_files = True
if create_csv_files:
    system = 'mcconnachie2012'
    table_numbers = [1, 2, 3, 4, 5]

    base_path = f"/Users/ayelland/Research/metal-poor-stars/spag/spag/data/galaxy_properties/{system}/"
    readme_path = os.path.join(base_path, "ReadMe")

    for n in table_numbers:
        table_path = os.path.join(base_path, f"table{n}.dat")
        csv_path = os.path.join(base_path, f"table{n}.csv")

        if os.path.exists(table_path) and os.path.exists(readme_path):
            try:
                table_data = ascii.read(
                    table_path,
                    format='cds',
                    readme=readme_path,
                    guess=False,
                    fast_reader=False,
                    fill_values=[('-', 'nan'), ('--', 'nan'), ('...', 'nan')]
                )
                table_data = table_data.to_pandas()
                table_data.to_csv(csv_path, index=False)
                print(f"✅ Converted table{n} to CSV.")
            except Exception as e:
                print(f"❌ Failed to read table{n}: {e}")
        else:
            print(f"⚠️  Missing table{n}.dat or ReadMe file for {system}.")


✅ Converted table1 to CSV.
❌ Failed to read table2: Column V(MW) failed to convert: invalid literal for int() with base 10: 'nan'
✅ Converted table3 to CSV.
✅ Converted table4 to CSV.
✅ Converted table5 to CSV.


### HMS/DMS $\leftrightarrow$ Degrees

In [ ]:
## Convert coordinates between degrees and hms/dms format

data = '''
242.63458901020093, -8.260693273003724
226.25888112673744, -13.738953843436517
272.43169186461097, -40.86888561990257
'''

def convert_deg_to_hmsdms(data):
    """
    Convert the given data string of coordinates from degrees to hms/dms format.
    """
    coords = []
    for line in data.strip().split('\n'):
        ra_deg, dec_deg = map(float, line.split(','))
        ra_hms = scoord.ra_deg_to_hms(ra_deg, precision=2)
        dec_dms = scoord.dec_deg_to_dms(dec_deg, precision=2)
        coords.append((ra_hms, dec_dms))
        # print(f"{ra_hms}, {dec_dms}")
    return coords

def convert_hmsdms_to_deg(data):
    """
    Convert the given data string of coordinates from degrees to hms/dms format.
    """
    coords = []
    for line in data.strip().split('\n'):
        ra_hms, dec_hms = map(str, line.split(','))
        ra_deg = scoord.ra_hms_to_deg(ra_hms, precision=6)
        dec_deg = scoord.dec_dms_to_deg(dec_hms, precision=6)
        coords.append((ra_deg, dec_deg))
        # print(f"{ra_deg}, {dec_deg}")

    return coords

# Convert the coordinates
try:
    txt = convert_deg_to_hmsdms(data)
except:
    try:
        txt = convert_hmsdms_to_deg(data)
    except Exception as e:
        print(f"Error converting coordinates: {e}")
        sys.exit(1)

print("Converted Coordinates:\n")
for c in txt:
    print(f'({c[0]} , {c[1]})')

Converted Coordinates:

(16:10:32.30 , -08:15:38.50)
(15:05:02.13 , -13:44:20.23)
(18:09:43.61 , -40:52:07.99)
